# Suivi d'Objets avec YOLOv8
Détection et suivi de tasses dans une séquence vidéo

## 2. Configuration initiale

In [17]:
import os
import cv2
import numpy as np
from scipy.optimize import linear_sum_assignment
from ultralytics import YOLO
import matplotlib.pyplot as plt
import shutil
import subprocess
# Configurations
FRAMES_DIR = './TP3_data/MOT17/MOT17/train/MOT17-02-FRCNN/img1'
INIT_FILE = './TP3_data/MOT17/MOT17/train/MOT17-02-FRCNN/gt/gt.txt'
OUTPUT_FILE = 'results.txt'
IOU_THRESHOLD = 0.6
MAX_AGE = 5
CLASS_NAME = 'cup'

## 3. Vérification des données

In [18]:
# Afficher les 5 premières frames
plt.figure(figsize=(15, 5))
for i in range(5):
    path = os.path.join(FRAMES_DIR, f'frame{i}.jpg')
    if os.path.exists(path):
        img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
        plt.subplot(1, 5, i+1)
        plt.imshow(img)
        plt.title(f'Frame {i}')
plt.tight_layout()
plt.show()

<Figure size 1500x500 with 0 Axes>

## 4. Classe de suivi

In [19]:
class Track:
    def __init__(self, track_id, bbox):
        self.id = track_id
        self.bbox = bbox  # [x1, y1, x2, y2]
        self.age = 0
    
    def update(self, new_bbox):
        self.bbox = new_bbox
        self.age = 0
    
    def predict(self):
        self.age += 1

## 5. Initialisation du modèle

In [20]:
# Charger le modèle YOLOv8
model = YOLO('yolo11n.pt')

# Vérifier la classe 'cup'
class_id = None
for k, v in model.names.items():
    if v == CLASS_NAME:
        class_id = k
        break
    elif v == 'cups':
        class_id = k
        break
        
if class_id is None:
    raise ValueError(f"Classe '{CLASS_NAME}' non trouvée dans le modèle")

## 6. Traitement principal

In [21]:
# Initialiser les tracks
tracks = []
with open(INIT_FILE, 'r') as f:
    for line in f:
        parts = list(map(int, line.strip().split()))
        tracks.append(Track(parts[1], [parts[2], parts[3], parts[2]+parts[4], parts[3]+parts[5]]))

# Lister toutes les frames disponibles 
frame_files = sorted([f for f in os.listdir(FRAMES_DIR) if f.lower().startswith('frame') and f.lower().endswith('.jpg')],key=lambda x: int(x[5:-4].lstrip('0') or 0))

# Journal des résultats
# Journal des résultats
with open(OUTPUT_FILE, 'w') as results_file:
    for frame_num, frame_file in enumerate(frame_files):
        # Charger l'image
        img = cv2.imread(os.path.join(FRAMES_DIR, frame_file))
        
        # Détection YOLOv8
        detections = []
        if frame_num > 0:
            # Utiliser un seuil de confiance à 0.5 et filtrer par classe cup
            results = model.predict(img, conf=0.5, classes=[class_id], verbose=False)
            # Parcourir les détections en récupérant les scores
            for box, conf in zip(results[0].boxes.xyxy.cpu().numpy(), results[0].boxes.conf.cpu().numpy()):
                # Conversion des coordonnées en entiers
                x1, y1, x2, y2 = map(int, box)
                # Filtrage géométrique optionnel : on conserve les détections dont le ratio largeur/hauteur est raisonnable pour une tasse
                width = x2 - x1
                height = y2 - y1
                if height > 0:
                    ratio = width / height
                    # On garde uniquement les détections avec un ratio entre 0.5 et 2.0
                    if ratio < 0.8 or ratio > 2.0:
                        continue
                detections.append([x1, y1, x2, y2])
        
        # Matrice IoU
        iou_matrix = np.zeros((len(tracks), len(detections)))
        for t, track in enumerate(tracks):
            for d, det in enumerate(detections):
                a = track.bbox
                b = det
                inter = max(0, min(a[2], b[2]) - max(a[0], b[0])) * max(0, min(a[3], b[3]) - max(a[1], b[1]))
                union = (a[2]-a[0])*(a[3]-a[1]) + (b[2]-b[0])*(b[3]-b[1]) - inter
                iou_matrix[t, d] = inter / union if union > 0 else 0
        
        # Association des détections
        row, col = linear_sum_assignment(-iou_matrix)
        matched = set()
        
        # Mise à jour des tracks
        for t, d in zip(row, col):
            if iou_matrix[t, d] >= IOU_THRESHOLD:
                tracks[t].update(detections[d])
                matched.add(d)
        
        # Nouveaux tracks
        for d in range(len(detections)):
            if d not in matched and frame_num > 0:
                new_id = max(t.id for t in tracks) + 1 if tracks else 1
                tracks.append(Track(new_id, detections[d]))
        
        # Vieillissement des tracks
        for track in tracks:
            track.predict()
        tracks = [t for t in tracks if t.age <= MAX_AGE]
        
        # Écrire les résultats
        for track in tracks:
            x1, y1, x2, y2 = track.bbox
            results_file.write(f"{frame_num} {track.id} {x1} {y1} {x2-x1} {y2-y1}\n")
        
        # Afficher le progrès
        if frame_num % 50 == 0:
            print(f"Traitement: {frame_num}/{len(frame_files)} frames")

ValueError: invalid literal for int() with base 10: '1,1,912,484,97,109,0,7,1'

## 7. Visualisation des résultats

In [ ]:
output_video = 'result_video.avi'
fps = 20  # adjust fps as needed

# Read the tracking results from results.txt
# Each line format: frame_num track_id x y width height
tracking_results = {}
with open(OUTPUT_FILE, 'r') as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) < 6:
            continue
        frame_num = int(parts[0])
        track_id = int(parts[1])
        x = int(parts[2])
        y = int(parts[3])
        w = int(parts[4])
        h = int(parts[5])
        tracking_results.setdefault(frame_num, []).append((track_id, x, y, w, h))

# Get the sorted list of frame files
frame_files = sorted(
    [f for f in os.listdir(FRAMES_DIR) if f.lower().startswith('frame') and f.lower().endswith('.jpg')],
    key=lambda x: int(x[5:-4].lstrip('0') or 0)
)

if not frame_files:
    raise ValueError("No frame images found in the directory.")

# Get frame dimensions from the first frame
first_frame = cv2.imread(os.path.join(FRAMES_DIR, frame_files[0]))
height, width, _ = first_frame.shape

# Initialize the video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_writer = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

# Process each frame, overlay tracking results, and write to the video
for idx, frame_file in enumerate(frame_files):
    frame = cv2.imread(os.path.join(FRAMES_DIR, frame_file))
    
    # Dessiner les boxes de suivi si disponibles pour la frame en cours
    if idx in tracking_results:
        for (track_id, x, y, w, h) in tracking_results[idx]:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f"ID {track_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0, 255, 0), 2)
    
    video_writer.write(frame)

video_writer.release()
print("Vidéo sauvegardée sous", output_video)


Vidéo sauvegardée sous result_video.avi


## 8. Evaluation avec TrackEval

Ici, nous préparons le fichier de résultats au format MOTChallenge complet et organisons
le dossier de sortie pour TrackEval. Vous devrez adapter le nom de la séquence et du tracker.

Par convention, nous utilisons :
- Un fichier de résultats au format MOT dans 'results.txt' (déjà généré ci-dessus)
- Nous le copions dans un fichier nommé results_MOT.txt (en respectant le format MOT à 10 colonnes)
- La structure attendue par TrackEval sera :
    TrackEval/data/trackers/mot_challenge/MOT17-train/<NomTracker>/<NomSequence>.txt

Vous devez également disposer du dossier des vérités terrain pour MOT17 (ou MOT20) 
dans TrackEval/data/gt/mot_challenge/MOT17-train/<NomSequence>/gt.txt

Réécriture optionnelle : ici nous copions directement le fichier OUTPUT_FILE en 
considérant que nous avons déjà écrit les 10 colonnes au bon format.

In [ ]:
result_mot_file = 'results_MOT.txt'
shutil.copy(OUTPUT_FILE, result_mot_file)
print(f"Fichier de résultats au format MOTChallenge généré : {result_mot_file}")

# Définir le nom de la séquence et du tracker (à ajuster)
sequence_name = "MOT17-01"    # Remplacez par le nom réel de la séquence utilisée
tracker_name = "MonTracker"   # Nom de votre tracker

# Organiser la structure de dossier pour TrackEval
# Exemple de structure pour MOT17-train :
output_tracker_dir = os.path.join("TrackEval", "data", "trackers", "mot_challenge", "MOT17-train", tracker_name)
os.makedirs(output_tracker_dir, exist_ok=True)

# Copier le fichier de résultats dans le dossier attendu par TrackEval
tracker_result_file = os.path.join(output_tracker_dir, f"{sequence_name}.txt")
shutil.copy(result_mot_file, tracker_result_file)
print(f"Résultats copiés vers {tracker_result_file}")

# Appel de l'évaluation TrackEval via un script
# Nous supposons que le script d'évaluation se trouve dans 'TrackEval/scripts/run_mot_challenge.py'
trackeval_script = os.path.join("TrackEval", "scripts", "run_mot_challenge.py")
benchmark = "MOT17"    # ou "MOT20" si applicable
split = "train"
cmd = [
    "python", trackeval_script,
    "--BENCHMARK", benchmark,
    "--SPLIT_TO_EVAL", split,
    "--TRACKERS_TO_EVAL", tracker_name,
    "--METRICS", "HOTA"
]
print("Lancement de l'évaluation avec TrackEval...")
subprocess.run(cmd)

Fichier de résultats au format MOTChallenge généré : results_MOT.txt
Résultats copiés vers TrackEval\data\trackers\mot_challenge\MOT17-train\MonTracker\MOT17-01.txt
Lancement de l'évaluation avec TrackEval...


CompletedProcess(args=['python', 'TrackEval\\scripts\\run_mot_challenge.py', '--BENCHMARK', 'MOT17', '--SPLIT_TO_EVAL', 'train', '--TRACKERS_TO_EVAL', 'MonTracker', '--METRICS', 'HOTA'], returncode=2)